# 영화 추천 시스템

## 1. 데이터 분비와 전처리

In [1]:
import os
import pandas as pd

rating_file_path=os.getenv('HOME') + '/aiffel/lms_project_dataset/EP9/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)
print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}') #orginal_data_size: 1000209, filtered_data_size: 836478
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}') # Ratio of Remaining Data is 83.63%

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [4]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/lms_project_dataset/EP9/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
# 평점과 제목 데이터프레임 합치기
ratings = ratings.join(movies.set_index('movie_id'), on='movie_id')
ratings.head()

,user_id,movie_id,count,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy


In [6]:
# 사용하지 않는 컬럼 제거
ratings.drop(columns=['timestamp', 'genre'], inplace=True)
ratings.head()

,user_id,movie_id,count,title
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975)
1,1,661,3,James and the Giant Peach (1996)
2,1,914,3,My Fair Lady (1964)
3,1,3408,4,Erin Brockovich (2000)
4,1,2355,5,"Bug's Life, A (1998)"


## 2. 데이터 분석

In [7]:
# ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [8]:
#rating에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [9]:
# ratings에 있는 유니크한 영화 제목 수
ratings['title'].nunique()

3628

In [10]:
# 가장 인기 있는 영화 30개
movie_count = ratings.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

In [11]:
# 유저별 몇 편의 영화를 봤는지에 대한 통계
user_count = ratings.groupby('user_id')['movie_id'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: movie_id, dtype: float64

## 3. 내가 선호하는 영화 추가

In [12]:
# 영화 제목 키워드로 검색
movies[movies['title'].str.lower().str.contains('matrix', regex=False)]

,movie_id,title,genre
2502,2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller


In [13]:
movies[movies['title'].str.lower().str.contains('mission', regex=False)]

,movie_id,title,genre
642,648,Mission: Impossible (1996),Action|Adventure|Mystery
2676,2745,"Mission, The (1986)",Drama
3285,3354,Mission to Mars (2000),Sci-Fi
3554,3623,Mission: Impossible 2 (2000),Action|Thriller
3817,3887,Went to Coney Island on a Mission From God... ...,Drama


In [14]:
movies[movies['title'].str.lower().str.contains('batman', regex=False)]

,movie_id,title,genre
151,153,Batman Forever (1995),Action|Adventure|Comedy|Crime
588,592,Batman (1989),Action|Adventure|Crime|Drama
1356,1377,Batman Returns (1992),Action|Adventure|Comedy|Crime
1522,1562,Batman & Robin (1997),Action|Adventure|Crime
3144,3213,Batman: Mask of the Phantasm (1993),Animation|Children's


In [15]:
# 내가 좋아하는 영화 5편의 id와 title 리스트 만들기
my_favorite_id = [2571, 592, 3190, 153, 648]

my_favorite_title = []
for i in my_favorite_id:
    my_favorite_title.extend(list(movies[movies['movie_id'] == i]['title']))

# 'cws'라는 user_id가 위 5편의 영화를 5번씩 봤다고 가정
my_movielist = pd.DataFrame({'user_id': ['cws']*5, 'movie_id': my_favorite_id, 'count': [5]*5, 'title': my_favorite_title})
my_movielist

,user_id,movie_id,count,title
0,cws,2571,5,"Matrix, The (1999)"
1,cws,592,5,Batman (1989)
2,cws,3190,5,Supernova (2000)
3,cws,153,5,Batman Forever (1995)
4,cws,648,5,Mission: Impossible (1996)


In [16]:
if not ratings.isin({'user_id':['cws']})['user_id'].any():  # user_id에 'cws'라는 데이터가 없다면
    ratings = ratings.append(my_movielist, ignore_index=True)   # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 
ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,count,title
836473,6040,1090,3,Platoon (1986)
836474,6040,1094,5,"Crying Game, The (1992)"
836475,6040,562,5,Welcome to the Dollhouse (1995)
836476,6040,1096,4,Sophie's Choice (1982)
836477,6040,1097,4,E.T. the Extra-Terrestrial (1982)
836478,cws,2571,5,"Matrix, The (1999)"
836479,cws,592,5,Batman (1989)
836480,cws,3190,5,Supernova (2000)
836481,cws,153,5,Batman Forever (1995)
836482,cws,648,5,Mission: Impossible (1996)


## 4. indexing 후 CSR matrix 생성

In [17]:
# 고유한 유저와 영화의 개수
user_unique = ratings['user_id'].unique()
movie_unique = ratings['title'].unique()

# 고유한 값의 개수만큼 유저 id와 영화 제목을 indexing
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [18]:
# 데이터 컬럼 내 값을 indexing된 값으로 교체

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('title column indexing OK!!')
    ratings['title'] = temp_movie_data
else:
    print('title column indexing Fail!!')

ratings

user_id column indexing OK!!
title column indexing OK!!


,user_id,movie_id,count,title
0,0,1193,5,0
1,0,661,3,1
2,0,914,3,2
3,0,3408,4,3
4,0,2355,5,4
...,...,...,...,...
836478,6039,2571,5,124
836479,6039,592,5,527
836480,6039,3190,5,2133
836481,6039,153,5,515


In [19]:
from scipy.sparse import csr_matrix

# CSR Matrix 생성
num_user = ratings['user_id'].nunique()
num_movie = ratings['title'].nunique()

csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.title)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 5. AlternatingLeastSquares 모델 훈련시키기

In [20]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [21]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [22]:
# als 모델은 input으로 item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [23]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

## 6. 훈련된 모델이 예측한 나의 선호도 파악

In [24]:
# 훈련된 모델이 만든 나의 벡터와 영화 벡터 구하기
cws, matx = user_to_idx['cws'], movie_to_idx['Matrix, The (1999)']
cws_vector, matx_vector = als_model.user_factors[cws], als_model.item_factors[matx]

In [25]:
cws_vector   # 나의 벡터

array([-0.7211637 ,  0.23890197,  0.20790431, -0.72571635,  0.5855989 ,
       -0.83064705,  1.3407241 , -0.08895839, -0.18831284,  0.04525429,
       -0.13167664, -0.7146999 ,  0.1343434 , -0.02323443, -0.789465  ,
       -0.29444927,  0.8348568 , -0.71125036,  0.09124728,  0.77996486,
        0.20762119, -0.31008744,  0.20107771,  0.5240948 ,  0.7549425 ,
       -0.08209966,  0.13121246,  0.30836764,  0.2786591 ,  0.44326878,
        0.0892057 , -0.7646562 ,  0.1965401 ,  0.1662847 , -0.54807407,
       -0.6519531 , -0.13231234,  0.4311663 ,  0.17574964,  0.11796129,
        0.44904783, -0.3995107 , -0.55559295,  0.4591082 , -0.5738629 ,
        0.5012314 ,  0.15633348, -0.27003184, -0.17648084,  0.05376681,
       -0.14011036,  0.35819912,  0.14351448,  0.42570335,  0.25712648,
        0.7285956 , -0.17179291, -0.3782973 ,  0.00233465, -0.07543807,
        0.48118955,  0.24360432,  0.7195256 , -1.3080782 , -0.12865354,
        0.07400642,  0.11467803,  0.21548785, -0.07437008,  0.23

In [26]:
matx_vector   # 영화 메트릭스의 벡터

array([-0.01096374,  0.00280871,  0.01215492,  0.00257925,  0.01416674,
       -0.00815265,  0.04587696, -0.01301556,  0.02004626,  0.00359348,
        0.01529257, -0.00871057, -0.00017439, -0.0028451 , -0.01033052,
       -0.00211601, -0.01424479, -0.01461952,  0.02668404,  0.02485612,
       -0.03700313,  0.01856934, -0.01028928,  0.03124723,  0.02426998,
        0.00704676, -0.00648011,  0.02917073,  0.00456015,  0.00175451,
        0.01224783, -0.02588319, -0.01539507,  0.00354898, -0.00236797,
        0.00761902,  0.00451177,  0.05251401,  0.00654605, -0.00290504,
        0.00873528, -0.0188643 , -0.01939296,  0.00600884,  0.01696185,
        0.00415875, -0.00089003,  0.02749247, -0.00587109, -0.00826017,
       -0.00984663,  0.00189571,  0.01134439,  0.01384447,  0.01513917,
        0.01454544,  0.00783363, -0.00760035,  0.02084303, -0.00449192,
        0.00805842,  0.04487018,  0.01715656, -0.01002972, -0.00703073,
        0.00670333,  0.01155587, -0.01129463,  0.01675453,  0.02

In [27]:
# 나와 매트릭스의 내적 구하기 (선호 리스트에 있는 영화)
np.dot(cws_vector, matx_vector)

0.38927627

In [28]:
# 다른 영화에 대한 선호도 예측 (선호 리스트에 없는 영화)
bat = movie_to_idx['Batman Returns (1992)']
bat_vector = als_model.item_factors[bat]
np.dot(cws_vector, bat_vector)

0.35453686

## 7. 내가 좋아하는 영화와 비슷한 영화 추천

In [29]:
favorite_movie = 'Mission: Impossible (1996)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie   # (movie idx, 유사도) Tuple

[(58, 1.0),
 (138, 0.6118344),
 (179, 0.452431),
 (118, 0.4311657),
 (129, 0.42881235),
 (587, 0.4278951),
 (140, 0.4255899),
 (119, 0.42145503),
 (156, 0.41683605),
 (82, 0.4166236),
 (975, 0.41562814),
 (766, 0.4154524),
 (107, 0.41143313),
 (67, 0.39394248),
 (746, 0.38794148)]

In [30]:
# movie_to_idx 를 뒤집어, index로부터 movie 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['Mission: Impossible (1996)',
 'True Lies (1994)',
 'Rock, The (1996)',
 'Con Air (1997)',
 'Conspiracy Theory (1997)',
 'Batman Returns (1992)',
 'Clear and Present Danger (1994)',
 'Twister (1996)',
 'Mask of Zorro, The (1998)',
 'Lost World: Jurassic Park, The (1997)',
 'Young Sherlock Holmes (1985)',
 'GoldenEye (1995)',
 'Jurassic Park (1993)',
 'Gladiator (2000)',
 'Perfect Storm, The (2000)']

In [31]:
# 비슷한 영화를 알려주는 함수
def get_similar_movie(movie_title: str):
    movie_id = movie_to_idx[movie_title]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [32]:
get_similar_movie('Mask of Zorro, The (1998)')

['Mask of Zorro, The (1998)',
 'GoldenEye (1995)',
 'Mummy, The (1999)',
 'Rock, The (1996)',
 'True Lies (1994)',
 'Rumble in the Bronx (1995)',
 'Man in the Iron Mask, The (1998)',
 'Three Musketeers, The (1993)',
 'Tomorrow Never Dies (1997)',
 'Ghost and the Darkness, The (1996)']

In [33]:
get_similar_movie('X-Men (2000)')

['X-Men (2000)',
 'Titan A.E. (2000)',
 'Shaft (2000)',
 'Gladiator (2000)',
 'Gone in 60 Seconds (2000)',
 'Scary Movie (2000)',
 'Mission: Impossible 2 (2000)',
 'Nutty Professor II: The Klumps (2000)',
 'Hollow Man (2000)',
 'Perfect Storm, The (2000)']

## 8. 내가 가장 좋아할 만한 영화 추천

In [34]:
user = user_to_idx['cws']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(587, 0.35453686),
 (107, 0.3090283),
 (92, 0.30040076),
 (372, 0.27228835),
 (172, 0.26320958),
 (141, 0.25451347),
 (67, 0.25130376),
 (62, 0.24612902),
 (175, 0.2458659),
 (1943, 0.22574618),
 (64, 0.21383241),
 (188, 0.20119175),
 (82, 0.19327235),
 (200, 0.18620701),
 (60, 0.18582389),
 (117, 0.18497568),
 (746, 0.1801131),
 (156, 0.17503604),
 (201, 0.17297222),
 (471, 0.17182666)]

In [35]:
# 인덱스를 영화 제목으로 변환
[idx_to_movie[i[0]] for i in movie_recommended]

['Batman Returns (1992)',
 'Jurassic Park (1993)',
 'Terminator 2: Judgment Day (1991)',
 'X-Men (2000)',
 'Indiana Jones and the Last Crusade (1989)',
 'Fugitive, The (1993)',
 'Gladiator (2000)',
 'Total Recall (1990)',
 'Men in Black (1997)',
 'Batman & Robin (1997)',
 'Star Wars: Episode VI - Return of the Jedi (1983)',
 'Indiana Jones and the Temple of Doom (1984)',
 'Lost World: Jurassic Park, The (1997)',
 'Terminator, The (1984)',
 'Star Wars: Episode I - The Phantom Menace (1999)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Perfect Storm, The (2000)',
 'Mask of Zorro, The (1998)',
 'Who Framed Roger Rabbit? (1988)',
 'Star Trek: First Contact (1996)']

In [36]:
# 사용자가 기록을 남긴 데이터가 특정 영화 추천에 기여한 정도를 확인
recommended = movie_to_idx['Total Recall (1990)']
explain = als_model.explain(user, csr_data, itemid=recommended)

[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('Matrix, The (1999)', 0.2226329961911913),
 ('Mission: Impossible (1996)', 0.030260615086847676),
 ('Batman (1989)', -0.00233972343958634),
 ('Batman Forever (1995)', -0.002639895653723451),
 ('Supernova (2000)', -0.006798994337025721)]

## 프로젝트 정리 
- MF: 추천시스템에 대한 기본 원리를 공부하면서 매트릭스 연산의 강력함을 알게 된 것고, 간단한 구조를 생각함으로써 많은 가치를 창출한다는 점에서 인상깊었다. 
- CSR Matrix생성: 데이타 공간을 효율적을 쓸 수 있는 데이터 구조를 사용함으로써 컴퓨터의 자원을 효율적으로 쓸 수 있는 방법을 배울 수 있어서 좋았다. 
- 결과분석: 내가 선호한 영화의 점수가 0.4정도라서 높은 편은 아니었던 것 같다. 내가 선호하는 영화는 아니었지만, 어느정도 유사성이 있다고 판단된 영화(batman시리즈)를 넣었을때는 점수(0.35)가 낮지 않게 나와서 추천 모델이 잘 판단하고 있다는 생각을 함.